## 모듈 임포트

In [1]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

## 스크래핑

In [2]:
url = "https://www.melon.com/chart/hot100/index.htm"
header = {"User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'}

response = requests.get(url, headers=header)
print(response)

<Response [200]>


In [3]:
soup = BeautifulSoup(response.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html lang="ko">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <title>
   멜론차트&gt;HOT100&gt;멜론
  </title>
  <meta content="음악서비스, 멜론차트, 멜론TOP100, 최신음악, 인기가요, 뮤직비디오, 앨범, 플레이어, 스트리밍, 다운로드, 아티스트플러스, 아티스트채널" name="keywords"/>
  <meta content="No.1 뮤직플랫폼 멜론! 최신 트렌드부터 나를 아는 똑똑한 음악추천까지!" name="description"/>
  <meta content="ee85ff6db1fa8f2226bcb671ecb2bcdbcffb6f8b" name="naver-site-verification"/>
  <meta content="q4tbTQhmxa4La3OdNLsNOCxrJ_WV6lUlBFrFW4-HqQc" name="google-site-verification"/>
  <meta content="357952407588971" property="fb:app_id"/>
  <meta content="Melon" property="og:title"/>
  <meta content="https://cdnimg.melon.co.kr/resource/image/web/common/logo_melon142x99.png" property="og:image"/>
  <meta content="음악이 필요한 순간, 멜론" property="og:description"/>
  <meta content="http://www.melon.com/chart/hot100/index.htm" property="og:url"/>
  <meta content="website" proper

### 상세 페이지 링크
- javascript:melon.link.goSongDetail('37657039') # 뒤에 숫자를 붙이면 됨
- "https://www.melon.com/song/lyrics.htm?songId=37657039"

In [4]:
tr_tags = soup.select('tbody tr')

rows = []

for tr_tag in tr_tags:
    
    td_tags = tr_tag.select('td')
    
    rank = td_tags[1].select_one('span.rank').get_text()  # 순위
    
    link = td_tags[4].select_one('a')['href']         # 상세정보로 가는 링크(javascrip 링크)
    song_id_start_index = link.find("(") + 2
    song_id_final_index = link.find(")") - 1
    song_id = link[song_id_start_index:song_id_final_index]
    detail_link = f"https://www.melon.com/song/lyrics.htm?songId={song_id}"  # 상세정보로 가는 url

    song_title = td_tags[5].select_one('div.rank01 > span').get_text().strip() # 곡 이름

    artist_names = ''
    a_tags = td_tags[5].select('div.rank02 > a')
    for a_tag in a_tags:
        artist_names += a_tag.get_text() + ", "
    artist_names = artist_names[:-2]         # 가수 이름

    album_name = td_tags[6].select_one('div.rank03 > a').get_text()            # 앨범 이름

    rows.append([rank, detail_link, song_title, artist_names, album_name])

rows

[['1',
  'https://www.melon.com/song/lyrics.htm?songId=37657039',
  'Small girl (feat. 도경수(D.O.))',
  '이영지',
  '16 Fantasy'],
 ['2',
  'https://www.melon.com/song/lyrics.htm?songId=37524037',
  'Supernova',
  'aespa',
  'Armageddon - The 1st Album'],
 ['3',
  'https://www.melon.com/song/lyrics.htm?songId=37563682',
  'How Sweet',
  'NewJeans',
  'How Sweet'],
 ['4',
  'https://www.melon.com/song/lyrics.htm?songId=37390939',
  '소나기',
  '이클립스 (ECLIPSE)',
  '선재 업고 튀어 OST Part 1'],
 ['5',
  'https://www.melon.com/song/lyrics.htm?songId=37491072',
  '온기',
  '임영웅',
  '온기'],
 ['6',
  'https://www.melon.com/song/lyrics.htm?songId=37464459',
  'Bubble Gum',
  'NewJeans',
  'How Sweet'],
 ['7',
  'https://www.melon.com/song/lyrics.htm?songId=37491071',
  'Home',
  '임영웅',
  '온기'],
 ['8',
  'https://www.melon.com/song/lyrics.htm?songId=37373234',
  '고민중독',
  'QWER',
  "1st Mini Album 'MANITO'"],
 ['9',
  'https://www.melon.com/song/lyrics.htm?songId=37524038',
  'Armageddon',
  'aespa',
  'Armaged

In [5]:
# 상세 페이지 url 가져오기
url_infos = []
for row in rows:
    url_info = row[1]
    url_infos.append(url_info)

print(len(url_infos))
url_infos[:5]

100


['https://www.melon.com/song/lyrics.htm?songId=37657039',
 'https://www.melon.com/song/lyrics.htm?songId=37524037',
 'https://www.melon.com/song/lyrics.htm?songId=37563682',
 'https://www.melon.com/song/lyrics.htm?songId=37390939',
 'https://www.melon.com/song/lyrics.htm?songId=37491072']

In [9]:
# 상세 페이지에서 상세 정보 스크래핑
row_for_extend = []
for url in url_infos:
    response = requests.get(url, headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    released_day = soup.select('dl.list dd')[1].get_text()  # 발매일
    genre = soup.select('dl.list dd')[2].get_text()   # 장르

    write = ''   # 작사
    compose = '' # 작곡
    arrange = '' # 편곡
    li_tags = soup.select('div.section_prdcr > ul > li')  # 작사/작곡
    for li_tag in li_tags:
        name = li_tag.select_one('div.entry a').get_text()
        role = li_tag.select_one('div.meta > span').get_text()
    
        if role == "작사": write += name + ", "
        if role == "작곡": compose += name + ", "
        if role == "편곡": arrange += name + ", "
    
    if write != "" and write[-1] == " ": write = write[:-2]
    if compose != "" and compose[-1] == " ": compose = compose[:-2]
    if arrange != "" and arrange[-1] == " ": arrange = arrange[:-2]
    
    print([released_day, genre, write, compose, arrange])
    row_for_extend.append([released_day, genre, write, compose, arrange])
    
row_for_extend

['2024.06.21', '랩/힙합', '이영지', 'PEEJAY, 이영지', 'PEEJAY']
['2024.05.13', '댄스', 'KENZIE', 'KENZIE, Paris Alexa, Dwayne “Dem Jointz” Abernathy Jr.', 'Dem Jointz']
['2024.05.24', '댄스', 'Gigi, Sarah Aarons, Elvira Anderfjard, Oscar Scheller, Stella Bennett, Tove Burman, 다니엘(DANIELLE)', '250, Sarah Aarons, Elvira Anderfjard, Oscar Scheller, Stella Bennett, Tove Burman', '250']
['2024.04.08', '발라드, 국내드라마', '한성호, 수윤 (Sooyoon)', '한성호, 박수석, Moon Kim (Room01)', '정진욱']
['2024.05.06', '발라드', '임영웅, 김이나', '임영웅, 김수형, 황선호', '물고기']
['2024.05.24', '댄스', 'Gigi, Oscar Bell, Sophie Simmons', '250, Oscar Bell, Sophie Simmons', '250']
['2024.05.06', '댄스', '임영웅, X-CHILD', 'X-CHILD', 'X-CHILD, 정수완']
['2024.04.01', '록/메탈', '이동혁, GESTURE, 김혜정, Elum, 마젠타', '이동혁, 홍훈기, Elum, GESTURE, 한아영', '이동혁, 한아영']
['2024.05.27', '댄스', '방혜현', 'EJAE, SUMIN (수민), Waker (153/Joombas), No Identity (노 아이덴티티)', 'No Identity (노 아이덴티티)']
['2024.04.26', '랩/힙합', '지코 (ZICO)', '지코 (ZICO), 은희영, No Identity (노 아이덴티티)', '지코 (ZICO), 은희영, No Identi

[['2024.06.21', '랩/힙합', '이영지', 'PEEJAY, 이영지', 'PEEJAY'],
 ['2024.05.13',
  '댄스',
  'KENZIE',
  'KENZIE, Paris Alexa, Dwayne “Dem Jointz” Abernathy Jr.',
  'Dem Jointz'],
 ['2024.05.24',
  '댄스',
  'Gigi, Sarah Aarons, Elvira Anderfjard, Oscar Scheller, Stella Bennett, Tove Burman, 다니엘(DANIELLE)',
  '250, Sarah Aarons, Elvira Anderfjard, Oscar Scheller, Stella Bennett, Tove Burman',
  '250'],
 ['2024.04.08',
  '발라드, 국내드라마',
  '한성호, 수윤 (Sooyoon)',
  '한성호, 박수석, Moon Kim (Room01)',
  '정진욱'],
 ['2024.05.06', '발라드', '임영웅, 김이나', '임영웅, 김수형, 황선호', '물고기'],
 ['2024.05.24',
  '댄스',
  'Gigi, Oscar Bell, Sophie Simmons',
  '250, Oscar Bell, Sophie Simmons',
  '250'],
 ['2024.05.06', '댄스', '임영웅, X-CHILD', 'X-CHILD', 'X-CHILD, 정수완'],
 ['2024.04.01',
  '록/메탈',
  '이동혁, GESTURE, 김혜정, Elum, 마젠타',
  '이동혁, 홍훈기, Elum, GESTURE, 한아영',
  '이동혁, 한아영'],
 ['2024.05.27',
  '댄스',
  '방혜현',
  'EJAE, SUMIN (수민), Waker (153/Joombas), No Identity (노 아이덴티티)',
  'No Identity (노 아이덴티티)'],
 ['2024.04.26',
  '랩/힙합',
  '지코 (ZICO

In [10]:
for i, row in enumerate(rows):
    row.extend(row_for_extend[i])
rows

[['1',
  'https://www.melon.com/song/lyrics.htm?songId=37657039',
  'Small girl (feat. 도경수(D.O.))',
  '이영지',
  '16 Fantasy',
  '2024.06.21',
  '랩/힙합',
  '이영지',
  'PEEJAY, 이영지',
  'PEEJAY'],
 ['2',
  'https://www.melon.com/song/lyrics.htm?songId=37524037',
  'Supernova',
  'aespa',
  'Armageddon - The 1st Album',
  '2024.05.13',
  '댄스',
  'KENZIE',
  'KENZIE, Paris Alexa, Dwayne “Dem Jointz” Abernathy Jr.',
  'Dem Jointz'],
 ['3',
  'https://www.melon.com/song/lyrics.htm?songId=37563682',
  'How Sweet',
  'NewJeans',
  'How Sweet',
  '2024.05.24',
  '댄스',
  'Gigi, Sarah Aarons, Elvira Anderfjard, Oscar Scheller, Stella Bennett, Tove Burman, 다니엘(DANIELLE)',
  '250, Sarah Aarons, Elvira Anderfjard, Oscar Scheller, Stella Bennett, Tove Burman',
  '250'],
 ['4',
  'https://www.melon.com/song/lyrics.htm?songId=37390939',
  '소나기',
  '이클립스 (ECLIPSE)',
  '선재 업고 튀어 OST Part 1',
  '2024.04.08',
  '발라드, 국내드라마',
  '한성호, 수윤 (Sooyoon)',
  '한성호, 박수석, Moon Kim (Room01)',
  '정진욱'],
 ['5',
  'https://www

In [11]:
columns = ["순위", "상세페이지", "곡 이름", "가수 이름", "앨범 이름", "발매일", "장르", "작사", "작곡", "편곡"]
df_melon = pd.DataFrame(data=rows, columns=columns)
df_melon = df_melon[["순위", "곡 이름", "가수 이름", "앨범 이름", "발매일", "장르", "작사", "작곡", "편곡", "상세페이지"]]
df_melon.head(10)

,순위,곡 이름,가수 이름,앨범 이름,발매일,장르,작사,작곡,편곡,상세페이지
0,1,Small girl (feat. 도경수(D.O.)),이영지,16 Fantasy,2024.06.21,랩/힙합,이영지,"PEEJAY, 이영지",PEEJAY,https://www.melon.com/song/lyrics.htm?songId=3...
1,2,Supernova,aespa,Armageddon - The 1st Album,2024.05.13,댄스,KENZIE,"KENZIE, Paris Alexa, Dwayne “Dem Jointz” Abern...",Dem Jointz,https://www.melon.com/song/lyrics.htm?songId=3...
2,3,How Sweet,NewJeans,How Sweet,2024.05.24,댄스,"Gigi, Sarah Aarons, Elvira Anderfjard, Oscar S...","250, Sarah Aarons, Elvira Anderfjard, Oscar Sc...",250,https://www.melon.com/song/lyrics.htm?songId=3...
3,4,소나기,이클립스 (ECLIPSE),선재 업고 튀어 OST Part 1,2024.04.08,"발라드, 국내드라마","한성호, 수윤 (Sooyoon)","한성호, 박수석, Moon Kim (Room01)",정진욱,https://www.melon.com/song/lyrics.htm?songId=3...
4,5,온기,임영웅,온기,2024.05.06,발라드,"임영웅, 김이나","임영웅, 김수형, 황선호",물고기,https://www.melon.com/song/lyrics.htm?songId=3...
5,6,Bubble Gum,NewJeans,How Sweet,2024.05.24,댄스,"Gigi, Oscar Bell, Sophie Simmons","250, Oscar Bell, Sophie Simmons",250,https://www.melon.com/song/lyrics.htm?songId=3...
6,7,Home,임영웅,온기,2024.05.06,댄스,"임영웅, X-CHILD",X-CHILD,"X-CHILD, 정수완",https://www.melon.com/song/lyrics.htm?songId=3...
7,8,고민중독,QWER,1st Mini Album 'MANITO',2024.04.01,록/메탈,"이동혁, GESTURE, 김혜정, Elum, 마젠타","이동혁, 홍훈기, Elum, GESTURE, 한아영","이동혁, 한아영",https://www.melon.com/song/lyrics.htm?songId=3...
8,9,Armageddon,aespa,Armageddon - The 1st Album,2024.05.27,댄스,방혜현,"EJAE, SUMIN (수민), Waker (153/Joombas), No Iden...",No Identity (노 아이덴티티),https://www.melon.com/song/lyrics.htm?songId=3...
9,10,SPOT! (feat. JENNIE),지코 (ZICO),SPOT!,2024.04.26,랩/힙합,지코 (ZICO),"지코 (ZICO), 은희영, No Identity (노 아이덴티티)","지코 (ZICO), 은희영, No Identity (노 아이덴티티)",https://www.melon.com/song/lyrics.htm?songId=3...


In [12]:
df_melon.to_excel('data/Melon_TOP100_20240705.xlsx', index=False)

In [13]:
df_melon[df_melon['곡 이름'] == '오래된 노래']

,순위,곡 이름,가수 이름,앨범 이름,발매일,장르,작사,작곡,편곡,상세페이지
15,16,오래된 노래,"허각, 임한별, 이무진, 이진성 (먼데이 키즈), 김희재, #안녕",오래된 노래,2024.06.16,발라드,"Egg1, Egg2, Egg3","Egg1, Egg2, Egg3","이도형 (AUG), 편준원(AUG)",https://www.melon.com/song/lyrics.htm?songId=3...
